In [134]:
# !! FINAL POLARITY CODE !!

# Google authentication key export
!export GOOGLE_APPLICATION_CREDENTIALS="/srv/Twitter2030/Data/acts-cloud-5-tljh-8e85a08eab1d-sakey.json"

from tweet_parser.tweet import Tweet
from tweet_parser.tweet_parser_errors import NotATweetError
import fileinput
import json
import string
import re
import time
t0 = time.time()
import nltk
# nltk.download('vader_lexicon')

from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# Imports the Google Cloud client library
from google.cloud import storage

# Create bucket client and authenticate via service account access key
storage_client = storage.Client.from_service_account_json('/srv/Twitter2030/Data/acts-cloud-5-tljh-8e85a08eab1d-sakey.json')

BUCKET_NAME = "allteamsmlb" # Do not change

team_names = ['02Angels', '03Athletics', '04Mariners', '05Astros', '06Rangers', '07Rockies', '08Padres', '09Dodgers', '10Giants', '11Diamondbacks',
              '12Cardinals', '13Reds', '14Cubs', '15Pirates', '16Brewers', '17Braves', '18Mets', '19Phillies', '20Marlins', '21Nationals', '22Indians',
              '23Twins', '24Royals', '25Tigers', '26WhiteSox', '27Yankees', '28Rays', '29RedSox', '30Orioles', '31BlueJays']

team_abbrevs = ['ANA', 'OAK', 'SEA', 'HOU', 'TEX', 'COL', 'SDP', 'LAD', 'SFG', 'AZ', 'STL', 'CIN', 'CHC', 'PIT', 'MIL', 'ATL', 'NYM', 'PHI', 
                'FLA', 'WSN', 'CLE', 'MIN', 'KCR', 'DET', 'CHW', 'NYY', 'TBD', 'BOS', 'BAL', 'TOR']

def clean_tweets(tweet):
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)

    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)

    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)

    # remove hashtags
    tweet = re.sub(r'#', '', tweet)
    
    return tweet

for team_name in team_names:
    print(team_name)
    team_abbrev = team_abbrevs[team_names.index(team_name)]
    dates = []
    win_loss = []

    for i in range(16, 20):
        with open(('Data/Team_WL/' + str(team_abbrev) + str(i) + '.csv'), newline='', encoding='utf-8-sig') as csvfile:
            reader = csv.reader(csvfile, delimiter=',', quotechar='|')
            for row in reader:
                date = row[0]
                if (date.startswith('d')):
                    continue
                if not date.startswith('1'):
                    date = '0' + date
                if not date[5] == '/':
                    date = date[:3] + '0' + date[3:]
                date = date[6:] + '-' + date[:2] + '-' + date[3:-5]
                dates.append(date)

    # Dictionaries: Key - Date    Value - Number of tweets of that sentiment
    neg_polarity_dict = {}
    neu_polarity_dict = {}
    pos_polarity_dict = {}
    com_polarity_dict = {}
 
    object_generator = storage_client.list_blobs(BUCKET_NAME, prefix=team_name, delimiter=None)

    for status_log_bucket_name in object_generator:
        client2 = storage.Client()
        BUCKET_NAME_TWO = client2.bucket(BUCKET_NAME)
        blob = BUCKET_NAME_TWO.blob(status_log_bucket_name.name)

        with blob.open('rt') as f:
            for line in f:
                try:
                    tweet_dict = json.loads(line)

                    tweet = Tweet(tweet_dict)

                    text = clean_tweets(tweet.all_text)

                    # Extracts date and verifies its a day the Sox play
                    date = tweet.created_at_string[0:10]

                    if date in dates:
                        scores = sia.polarity_scores(text) # Create a dict of scores

                        # Loop the dict of scores
                        for key in scores:
                            if key == 'neg':
                                if scores[key] == 0.0:
                                    if date in neg_polarity_dict.keys():
                                        neg_polarity_dict[date] += 1
                                    else:
                                        neg_polarity_dict[date] = 1
                            if key == 'neu':
                                if scores[key] == 0.0:
                                    if date in neu_polarity_dict.keys():
                                        neu_polarity_dict[date] += 1
                                    else:
                                        neu_polarity_dict[date] = 1
                            if key == 'pos':
                                if scores[key] == 0.0:
                                    if date in pos_polarity_dict.keys():
                                        pos_polarity_dict[date] += 1
                                    else:
                                        pos_polarity_dict[date] = 1
                            if key == 'compound':
                                if scores[key] == 0.0:
                                    if date in com_polarity_dict.keys():
                                        com_polarity_dict[date] += 1
                                    else:
                                        com_polarity_dict[date] = 1

                except (json.JSONDecodeError, NotATweetError):
                    pass
                
    # Writes to text files
    with open('Output/Polarity/' + team_abbrev + '_negative.txt', 'w') as new_file: 
        for key, value in neg_polarity_dict.items(): 
            new_file.write('%s : %d\n' % (key, value))

    with open('Output/Polarity/' + team_abbrev + '_neutral.txt', 'w') as new_file: 
        for key, value in neu_polarity_dict.items(): 
            new_file.write('%s : %d\n' % (key, value))

    with open('Output/Polarity/' + team_abbrev + '_positive.txt', 'w') as new_file: 
        for key, value in pos_polarity_dict.items(): 
            new_file.write('%s : %d\n' % (key, value))

    with open('Output/Polarity/' + team_abbrev + '_compound.txt', 'w') as new_file: 
        for key, value in com_polarity_dict.items(): 
            new_file.write('%s : %d\n' % (key, value))
                    
t1 = time.time()
m, s = divmod(round(t1 - t0), 60)

print('Runtime - ', str(m), 'm ', str(s), 's')
print('El Fin')

02Angels


KeyboardInterrupt: 